# 4주차
## Regularized Linear Regression and Bias vs Variance

In [ ]:
import os, sys
import numpy as np
from matplotlib import pyplot

# scipy내에 있는 최적화기능이 있는 모듈
from scipy import optimize
from scipy.optimize import minimize

# MATLAB의 mat 형식의 파일을 읽기 위해 사용
from scipy.io import loadmat

# 제출을 하기 위한 라이브러리
sys.path.append('../src/')
from utils import submit
from utils import help_me
from answer import trainLinearReg, featureNormalize, plotFit
# matplotlib가 노트북에 플롯을 포함하도록 지시
%matplotlib inline

<a id="section1"></a>
## 1 Regularized Linear Regression

이번 문제에서는, 저수지의 물의 양이 달라짐에 따라 댐에서 나오는 물의 양이 어떻게 변하는지 regularized linear regression을 통해 예측해 봅시다. bias와 variance의 변화를 보면서 알맞은 lambda값을 찾아 볼 것입니다. 

### 1.1 Visualizing the dataset

우선 데이터가 어떻게 구성되어 있는지 봅시다. 

- theta를 학습시키기 위한 training set: `X`, `y`
- lambda값을 조정시키기 위한 cross validation set: `Xval`, `yval`
- 모델의 성능을 평가하기 위한 test set: `Xtest`, `ytest`

우선 다음 셀을 실행시켜서 training set을 시각화 해보세요. 그리고 나서 linear regression을 사용해서 training set에 직선을 fitting 시킬 것입니다. 그 후에는, polynomial regression을 활용해서 training set에 더 알맞은 선을 fitting 시킬 것입니다. 

In [ ]:
# 데이터 불러오기, dict 형태로 저장됩니다. 
data = loadmat(os.path.join('Data', 'ex5data1.mat'))

In [ ]:
# 데이터 나누기
X, y = data['X'], data['y'][:, 0]
Xtest, ytest = data['Xtest'], data['ytest'][:, 0]
Xval, yval = data['Xval'], data['yval'][:, 0]

print('X:', X.shape)
print('y:', y.shape)
print('Xval:', Xval.shape)
print('yval:', yval.shape)

In [ ]:
# training set의 sample 개수
m = y.size

In [ ]:
# training set 시각화 
pyplot.plot(X, y, 'ro', ms=10, mec='k', mew=1)
pyplot.xlabel('Change in water level (x)')
pyplot.ylabel('Water flowing out of the dam (y)');

### 1.2 Regularized linear regression cost function
Regularized linear regression의 cost function은 다음과 같습니다.:

$$ J(\theta) = \frac{1}{2m} \left( \sum_{i=1}^m \left( h_\theta\left( x^{(i)} \right) - y^{(i)} \right)^2 \right) + \frac{\lambda}{2m} \left( \sum_{j=1}^n \theta_j^2 \right)$$

$\lambda$가 regularization의 정도를 결정합니다. 그렇게 함으로써 overfitting을 방지합니다. lambda가 증가할 수록 cost function은 패널티를 더 많이 받게 됩니다. 참고로 $\theta_0$은 상수항에 대한 파라미터이기 때문에 패널티를 부여하지 않습니다. 

다음 셀에 있는 `linearRegCostFunction`을 완성해보세요. 목표는 regularized linear regression의 cost를 계산하는 것입니다. 가능한 for문을 사용하지 말고 행렬로 한번에 계산해 보세요. 
<a id="linearRegCostFunction"></a>

간단한 예제 theta를 통해 연습한 후 진짜 함수를 정의해보겠습니다  
차례대로 따라간 후 연습한 문항을 통해 함수를 정의해보세요!

In [ ]:
# 연습을 위해 X데이터에 상수항을 추가하겠습니다. 프린트하시고 형태 확인해보세욥
X_exer = np.concatenate([np.ones((m, 1)), X], axis=1)
print(X_exer)

In [ ]:
# theta와 데이터 X_exer가 주어졌을 때 hypothesis 값을 구해보겠습니다
theta = np.random.rand(X_exer.shape[1])
hypothesis = None
print(hypothesis)

In [ ]:
# 앞서 구한 hypothesis 값을 이용해 J의 앞부분 (regularization 부분을 제외한) 을 먼저 구해보겠습니다
J_A = None
print(J_A)

In [ ]:
# 이제 J의 뒷부분을 구해보겠습니다. theta[0] 값은 cost function에 포함되지 않는걸 주의하세요!
# theta[0] 값을 제외하면 theta 값이 1개 밖에 없지만, 여러개가 있다고 생각하고 일반화해서 계산해주세요
lambda_ = 1
J_B = None
print(J_B)

지금까지 연습한 부분을 바탕으로 J 값을 구해주시면 됩니다!  

### 1.3 Regularized linear regression gradient

gradient는 다음과 같이 계산됩니다.: 


$$
\begin{align}
& \frac{\partial J(\theta)}{\partial \theta_0} = \frac{1}{m} \sum_{i=1}^m \left( h_\theta \left(x^{(i)} \right) - y^{(i)} \right) x_j^{(i)} & \qquad \text{for } j = 0 \\
& \frac{\partial J(\theta)}{\partial \theta_j} = \left( \frac{1}{m} \sum_{i=1}^m \left( h_\theta \left( x^{(i)} \right) - y^{(i)} \right) x_j^{(i)} \right) + \frac{\lambda}{m} \theta_j & \qquad \text{for } j \ge 1
\end{align}
$$


다음 셀에 있는 `lrgradient` 를 계산해보세요. 목표는 regularized linear regression의 grad를 구하는 것입니다.

마찬가지로 간단한 연습 후에 진짜 함수를 정의해 보겠습니다.  
theta[1] ~ 의 gradient를 구할 때 for문을 사용하지 않고 행렬 계산을 통해 진행해보겠습니다

In [ ]:
# 앞서 정의한 hypothesis, theta, X_exer 를 통해 theta[0] 의 gradient를 구해보겠습니다
# theta[0] 의 gradient는 theta값을 포함하지 않는것을 주의하세요
grad_exer = np.zeros(theta.size)
grad_exer[0] = None
grad_exer

In [ ]:
# 이제 theta[0]를 제외한 나머지 theta의 gradient를 구해야합니다
# 연습문항에는 theta[0]를 제외하면 theta 값이 1개 밖에 없지만, 여러개가 있다고 생각하고 일반화해서 계산해주세요
# 먼저 gradient의 앞부분(lambda_가 들어가는 부분을 제외한) 을 먼저 계산해보겠습니다.
grad_exer_A = None
grad_exer_A

In [ ]:
# 이제 뒷부분을 계산해보겠습니다
lambda_ = 1
grad_exer_B = None
grad_exer_B

이제 연습한 부분을 바탕으로 함수를 만들어 보겠습니다  
grad[1:] 는 앞서 연습한 부분에서 grad_exer_A 와 grad_exer_B를 더해주기만 하면 됩니다  

In [ ]:
def linearRegCostFunction(X, y, theta, lambda_=0.0):
    """
    regularized linear regression의 cost와 gradient을 계산하는 함수 
    
    Parameters
    ----------
    X : array_like
        (m x n + 1) 크기의 행렬. m은 sample의 개수이고, n은 
        상수항을 더하기 이전의 feature의 개수 
        
    y : array_like
        각 sample들의 label값
        크기는 (m, )
        
    theta : array_like
        모델의 파라미터.
        크기는 (n+1, )
    
    lambda_ : float, optional
        regularization 파라미터 
    
    Returns
    -------
    J : float
        cost 값
    
    grad : array_like
        theta별 gradient 값
        크기는 (n+1, )
        
    """
    # training 샘플 개수 
    m = y.size 

    J = 0
    grad = np.zeros(theta.shape)

    # ====================== YOUR CODE HERE ======================
    hypothesis = None
    J = None
    grad[0] = None 
    grad[1:] = None
    # ============================================================
    return J, grad

`theta` 를 `[1, 1]`로 초기화하고 다음 셀을 실행하면 303.993192가 나와야 할 것입니다. 

In [ ]:
theta = np.array([1, 1])
J, _ = linearRegCostFunction(np.concatenate([np.ones((m, 1)), X], axis=1), y, theta, 1)

print('Cost at theta = [1, 1] : {}'.format(round(J, 6)))
print('(This value should be about 303.993192)\n')

다음셀의 값은 [-15.303016, 598.250744]`가 나와야 할 것입니다. 

In [ ]:
theta = np.array([1, 1])
J, grad = linearRegCostFunction(np.concatenate([np.ones((m, 1)), X], axis=1), y, theta, 1)

print('Gradient at theta = [1, 1]:  [{:.6f}, {:.6f}] '.format(*grad))
print(' (this value should be about [-15.303016, 598.250744])\n')

답이 맞는지 확인해보세요.

In [ ]:
submit(linearRegCostFunction)

### Fitting linear regression

다음 셀을 실행시키면, 최적의 theta 값을 찾게 될 것입니다. `trainLinearReg`을 사용하면 저번주에 사용했던 `scipy`의 optimize 모듈을 사용해서 최적의 theta값을 찾게 됩니다. 

여기에서는 두개의 feature (상수항이랑 원래 feature)에 대해서만 학습을 진행하기 때문에, lambda를 0으로 두겠습니다. 왜냐하면 feature의 수가 너무 적어서 regularization을 해봤자 별 도움이 안됩니다. 

아래 코드를 실행하면 다음과 같은 그림이 나와야 할 것입니다. 

![](Figures/linear_fit.png)

보시면은 training set에는 비선형적인 패턴이 있어서, 직선으로는 잘 fitting 되지 않습니다. 2차원에서는 이렇기 시각화해서 확인할 수 있지만, 차원이 커질 수록 시각화 하기가 힘듭니다. 다음 단계에서는 learning curve를 사용하여 어떻게 모델을 조정시켜줄지 확인해봅시다. 

In [1]:
# add a columns of ones for the y-intercept
X_aug = np.concatenate([np.ones((m, 1)), X], axis=1)
theta = trainLinearReg(linearRegCostFunction, X_aug, y, lambda_=0)

#  Plot fit over the data
pyplot.plot(X, y, 'ro', ms=10, mec='k', mew=1.5)
pyplot.xlabel('Change in water level (x)')
pyplot.ylabel('Water flowing out of the dam (y)')
pyplot.plot(X, np.dot(X_aug, theta), '--', lw=2);

NameError: name 'np' is not defined

<a id="section3"></a>
## 2 Bias-variance

### 2.1 Learning Curves

learning curve는 training set의 샘플 수(x축)에 따라 training error와 cross validation error(y축)를 시각화합니다. `learning curve` 함수는 training error와 cross validation error를 반환하는 함수입니다. 

X[:i, :] 와 y[:i]를 통해 i개의 샘플을 활용해서 i번째 training error를 계산해보세요. cross validation error는 cross validation set에 있는 모든 샘플들을 활용해 계산하세요. 

theta를 구하기 위해서는 위에서 사용한 `utils.trainLinearReg()`함수를 사용하세요.

training error는 다음과 같이 정의됩니다. 

$$ J_{\text{train}} = \frac{1}{2m} \left[ \sum_{i=1}^m \left(h_\theta \left( x^{(i)} \right) - y^{(i)} \right)^2 \right] $$

여기서는 regularized term이 없는 것에 주의하세요. 즉 기존에 구축한 `linearRegCostFunction` 함수를 사용할 것이라면, lambda_를 0으로 줘야 합니다. 

i 번째 training error는 i개의 샘플에 대해서 계산하고, 반면 cross validation error는 항상 모든 cross validation set의 샘플들에 대해서 계산하는거 잊지마세요!!


In [ ]:
# 앞서 정의한 linearRegCostFunction를 통해 J 값을 구해보겠습니다
# X[:5, :], y[:5]에 써보세요! lambda 값이 0이여야 합니다
J_exer = None
J_exer

In [ ]:
def learningCurve(X, y, Xval, yval, lambda_=0):
    """
    training error와 cross validation error를 반환하는 함수 

    Parameters
    ----------
    X : array_like
        (m x n + 1) 크기의 행렬. m은 sample의 개수이고, n은 
        상수항을 더하기 이전의 feature의 개수 
        
    y : array_like
        각 sample들의 label값
        크기는 (m, )
        
    Xval : array_like
        (m_val x n + 1) 크기의 행렬. m_val은 sample의 개수이고, n은 
        상수항을 더하기 이전의 feature의 개수 
        
    yval : array_like
        각 validation sample들의 label값
        크기는 (m_val, )
        
    lambda_ : float, optional
        regularization 파라미터 
    
    Returns
    -------
    error_train : array_like
        크기 m인 벡터. 
        
    error_val : array_like
        크기 m인 벡터. 

    Instructions
    ------------
    error_train[i-1] = i개의 training sample을 통해 학습한 모델의 training error
    error_val[i-1] = i개의 training sample을 통해 학습한 모델의 cross validation error
    
    Notes
    -----
    
    - linearRegCostFunction을 활용해서 구현한다면은, training error와 cross validation error를 구할 때는
    lambda_값을 0으로 두세요. 반면, theta를 구할 때는 lambda_값을 그대로 반영해야 합니다. 
    
    Hint
    ----
    다음의 의사코드를 활용해보세요
     
           for i in range(1, m+1):
               # 1. X[:i, :]와 y[:i], 그리고 lambda_를 활용해서 theta 구하기, (utils.trainLinearReg() 함수 사용하기)
               # 2. training error, cross validation error 구하기
               # 3. error_train[i-1] 그리고 error_val[i-1]에다가 구한 값 저장하기 
               ....  
    """
    # Number of training examples
    m = y.size

    # You need to return these values correctly
    error_train = np.zeros(m)
    error_val   = np.zeros(m)

    # ====================== YOUR CODE HERE ======================
         
    for i in range(1, m+1):    
        # i개의 sample로 theta 학습
        theta = None

        # 학습된 theta로 예측값 산출하고, 실제값과 차이
        error_train[i-1] = None
        error_val[i-1] = None
        
    # =============================================================
    return error_train, error_val

아래 셀을 실행시키면, 다음과 같은 그림이 나와야 할 것입니다. 

![](Figures/learning_curve.png)

위와 같은 그림은, 모델의 bias가 클 경우 나타납니다. 즉 우리의 linear regression 모델이 너무 간단하기 때문에 data에 잘 fitting되지 않는 거죠. 다음 단계에서는 polynomial regression을 통해 data에 더 잘 fitting되는 모델을 찾아 볼 것입니다. 

In [ ]:
X_aug = np.concatenate([np.ones((m, 1)), X], axis=1)
Xval_aug = np.concatenate([np.ones((yval.size, 1)), Xval], axis=1)
error_train, error_val = learningCurve(X_aug, y, Xval_aug, yval, lambda_=0)

pyplot.plot(np.arange(1, m+1), error_train, np.arange(1, m+1), error_val, lw=2)
pyplot.title('Learning curve for linear regression')
pyplot.legend(['Train', 'Cross Validation'])
pyplot.xlabel('Number of training examples')
pyplot.ylabel('Error')
pyplot.axis([0, 13, 0, 150])

print('# Training Examples\tTrain Error\tCross Validation Error')
for i in range(m):
    print('  \t%d\t\t%f\t%f' % (i+1, error_train[i], error_val[i]))

답안을 제출해보세요 

In [ ]:
submit(learningCurve)

## 3 Polynomial regression

앞서 linear regression의 문제는 feature수가 너무 적다는 것이였죠. 그에 반면
polynomial regression의 hypothesis는 다음과 같습니다. 

$$
\begin{align}
h_\theta(x)  &= \theta_0 + \theta_1 \times (\text{waterLevel}) + \theta_2 \times (\text{waterLevel})^2 + \cdots + \theta_p \times (\text{waterLevel})^p \\
& = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots + \theta_p x_p
\end{align}
$$

 $x_1 = (\text{waterLevel})$, $x_2 = (\text{waterLevel})^2$ , $\cdots$, $x_p =
(\text{waterLevel})^p$, 이런식으로 생각을 해본다면은, 우리가 위에서 사용한 함수들을 그대로 적용할 수 있는 간단한 multiple linear regression 문제가 됩니다. 

데이터셋의 feature를 이렇게 늘리는 함수인 `polyFeatures`를 만들어 봅시다. 해당 함수는 크기 $m \times 1$인 training set $X$ 를 $m \times p$ 인 `X_poly`로 만들어 줍니다. i번째 column은 X를 i번 제곱한 값을 지니게 되는거죠.

np.power() 함수를 사용해서 문제를 풀어보세요. 

In [ ]:
# np.power() 함수는 array를 받아 array의 각 요소를 지정 횟수만큼 제곱시켜주는 함수입니다
A = np.array([2, 3, 5])
np.power(A, 3)

In [ ]:
# 현재 column 수가 1개인 X가 있습니다.
# X_poly에 x1 = x, x2 = x^3 인 컬럼을 2개 가지는 변수를 할당해 보세요
X_poly = np.zeros((X.shape[0], 2))
X_poly[:, 0] = None
X_poly[:, 1] = None

In [ ]:
def polyFeatures(X, p):
    """
    p승 만큼 데이터를 뻥튀기 시킨다. 
    
    Parameters
    ----------
    X : array_like
        크기 m인 벡터. m은 training sample의 개수 
    
    p : int
        뻥튀기 하고자 하는 power 값
    
    Returns 
    -------
    X_poly : array_like
        크기 m x p인 행렬 

        각 행의 값을 확인해보면 다음과 같음
        
        X_poly[i, :] = [X[i], X[i]**2, X[i]**3 ...  X[i]**p]
    
    """
    
    X_poly = np.zeros((X.shape[0], p))

    # ====================== YOUR CODE HERE ======================

    for i in range(p) :

        X_poly[:, i] = None

    # ============================================================
    return X_poly

해당 함수를 training set, cross validation set, test set에 적용해봅시다.

In [ ]:
p = 8

# feature 증가 시키고 normalize 해주기
X_poly = polyFeatures(X, p)
X_poly, mu, sigma = featureNormalize.test(X_poly)
X_poly = np.concatenate([np.ones((m, 1)), X_poly], axis=1)

# feature 증가 시키고 normalize 해주기
X_poly_test = polyFeatures(Xtest, p)
X_poly_test -= mu
X_poly_test /= sigma
X_poly_test = np.concatenate([np.ones((ytest.size, 1)), X_poly_test], axis=1)

# feature 증가 시키고 normalize 해주기
X_poly_val = polyFeatures(Xval, p)
X_poly_val -= mu
X_poly_val /= sigma
X_poly_val = np.concatenate([np.ones((yval.size, 1)), X_poly_val], axis=1)

print('Normalized Training Example 1:')
X_poly[0, :]

답안을 제출해보세요.

In [ ]:
submit(polyFeatures)

## 3.1 Learning Polynomial Regression

이제 theta를 학습시킨뒤 lambda가 0인 상태에서 시각화를 하게 되면 다음과 같은 그래프가 나옵니다.:

<table>
    <tr>
        <td><img src="Figures/polynomial_regression.png"></td>
        <td><img src="Figures/polynomial_learning_curve.png"></td>
    </tr>
</table>

training set에 정말 잘 fitting되는 것을 확인할 수 있습니다. 허나 cross validation set에 대해서는 error가 감소하다가 증가하는 것을 볼 수 있습니다. 
즉 training set에 overfitting 되었다는 의미인거죠. 또한 training error와 cross validation error 사이에 큰 gap이 있습니다. 이것은 high variance가 문제라는 뜻입니다. 

In [ ]:
lambda_ = 0
theta = trainLinearReg(linearRegCostFunction, X_poly, y,
                             lambda_=lambda_, maxiter=55)

# Plot training data and fit
pyplot.plot(X, y, 'ro', ms=10, mew=1.5, mec='k')

plotFit(polyFeatures, np.min(X), np.max(X), mu, sigma, theta, p)

pyplot.xlabel('Change in water level (x)')
pyplot.ylabel('Water flowing out of the dam (y)')
pyplot.title('Polynomial Regression Fit (lambda = %f)' % lambda_)
pyplot.ylim([-20, 50])

pyplot.figure()
error_train, error_val = learningCurve(X_poly, y, X_poly_val, yval, lambda_)
pyplot.plot(np.arange(1, 1+m), error_train, np.arange(1, 1+m), error_val)

pyplot.title('Polynomial Regression Learning Curve (lambda = %f)' % lambda_)
pyplot.xlabel('Number of training examples')
pyplot.ylabel('Error')
pyplot.axis([0, 13, 0, 100])
pyplot.legend(['Train', 'Cross Validation'])

print('Polynomial Regression (lambda = %f)\n' % lambda_)
print('# Training Examples\tTrain Error\tCross Validation Error')
for i in range(m):
    print('  \t%d\t\t%f\t%f' % (i+1, error_train[i], error_val[i]))

overfitting 문제, 즉 high variance 문제를 해결하기 위한 방법 중에 regularization term을 활용하는 방법이 있습니다. 아래 내용을 참고하세요. 

### 3.2 Optional (ungraded) exercise: Adjusting the regularization parameter

위의 셀에서 lambda_를 1 그리고 100으로 두고 각각 셀을 실행시켜 보세요. lambda_를 1로 두게 되면 fitting도 잘되고, learning curve도 비교적 적은 값으로 수렴하는 것을 볼 수 있을 것입니다. 즉 bias와 variance 간의 trade off가 잘 이루어 졌다는 것입니다. 아래의 그림과 같은 그래프가 나올 것입니다. 

<table>
    <tr>
        <td><img src="Figures/polynomial_regression_reg_1.png"></td>
        <td><img src="Figures/polynomial_learning_curve_reg_1.png"></td>
    </tr>
</table>

반면 lambda_가 100일 때는 fitting이 잘 안된 것을 볼 수 있습니다. 즉 패널티가 너무 많이 반영되었다는 뜻이죠. 

![](Figures/polynomial_regression_reg_100.png)


### 3.3 Selecting $\lambda$ using a cross validation set

이제 lambda를 결정하기 위한 함수를 구현해볼 것입니다. lambda가 변할 때마다 training set error와 cross validation error가 어떻게 변하는지 확인할 것입니다. 

`validationCurve` 함수는 사용한 lambda 값과, lambda값 별 training error와 cross validation error를 반환하는 함수입니다. 


In [ ]:
def validationCurve(X, y, Xval, yval):
   
    # 사용할 lambda 값들 
    lambda_vec = [0, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]

    error_train = np.zeros(len(lambda_vec))
    error_val = np.zeros(len(lambda_vec))

    # ====================== YOUR CODE HERE ======================

    for i in range(len(lambda_vec)):
        
        lambda_ = lambda_vec[i]
        
        theta = trainLinearReg(None, None, None, None) 
        
        # 학습된 theta로 예측값 산출하고, 실제값과 차이 계산
        # linearRegCostFunction()을 활용해보세요
        
        error_train[i], _ = None
        
        error_val[i], _ = None

    # ============================================================
    return lambda_vec, error_train, error_val

아래 셀을 실행시키면 다음과 같은 그래프가 나올 것입니다. 

![](Figures/cross_validation.png)

그래프를 확인해보면, cross validation error가 가장 작은 lambda의 값은 3인 것을 확인 할 수 있습니다. 

In [ ]:
def validationCurve(X, y, Xval, yval):

    lambda_vec = [0, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]
    error_train = np.zeros(len(lambda_vec))
    error_val = np.zeros(len(lambda_vec))

    for i in range(len(lambda_vec)):
        lambda_ = lambda_vec[i]
        theta = trainLinearReg(linearRegCostFunction, X, y, lambda_)
        error_train[i], _ = linearRegCostFunction(X, y, theta, 0)
        error_val[i], _ = linearRegCostFunction(Xval, yval, theta, 0)
    return lambda_vec, error_train, error_val

In [ ]:
lambda_vec, error_train, error_val = validationCurve(X_poly, y, X_poly_val, yval)

pyplot.plot(lambda_vec, error_train, '-o', lambda_vec, error_val, '-o', lw=2)
pyplot.legend(['Train', 'Cross Validation'])
pyplot.xlabel('lambda')
pyplot.ylabel('Error')

print('lambda\t\tTrain Error\tValidation Error')
for i in range(len(lambda_vec)):
    print(' %f\t%f\t%f' % (lambda_vec[i], error_train[i], error_val[i]))

결과를 제출해보세요.

In [ ]:
submit(validationCurve)